<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li></ul></div>

# LinkedIn Scraper

## Import Packages

In [137]:
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium import webdriver
from config import LinkedInProfile

## Functions

In [160]:
def sign_in(driver, li_profile):
    """
    Function takes a webdriver object and a LinkedInProfile object and uses 
    the credentials from the LinkedInProfile object to log the driver into LinkedIn.
    
    
    """
    
    driver.get('https://www.linkedin.com')
    driver.find_element_by_xpath("//nav/a[@class='nav__button-secondary']").click()

    time.sleep(3)

    username = driver.find_element_by_xpath("//input[@id='username']")
    password = driver.find_element_by_xpath("//input[@id='password']")

    username.send_keys(li_profile.username)
    password.send_keys(li_profile.password)

    driver.find_element_by_xpath("//button[@type='submit']").click()

def close_driver(driver):
    try:
        driver.close()
    except:
        pass
    driver.quit()
    
def click_all_see_more_buttons(driver):
    see_more_button_xpath = "//button[@class='pv-profile-section__see-more-inline pv-profile-section__text-truncate-toggle link link-without-hover-state']"
    show_more_elements = driver.find_elements_by_xpath(see_more_button_xpath)
    for button in show_more_elements:
        actions = ActionChains(driver)
        coordinates = button.location_once_scrolled_into_view # returns dict of X, Y coordinates
        driver.execute_script(f"window.scrollTo({coordinates['x']}, {coordinates['y']});")
        button.click()
    
def check_for_see_more_buttons(driver):
    see_more_button_xpath = "//button[@class='pv-profile-section__see-more-inline pv-profile-section__text-truncate-toggle link link-without-hover-state']"
    show_more_elements = driver.find_elements_by_xpath(see_more_button_xpath)
    if len(show_more_elements) > 0:
        return True
    else:
        return False
    
    

def scrape_profile(driver, profile_url):
    driver.maximize_window()
    profile_data = {}
    
    driver.get(profile_url)
    base_first_xpath = "//div[@class='application-outlet ']//div[@class='ph5 pb5']/div[@class='display-flex mt2']/div[@class='flex-1 mr5']"
    top_box_uls = driver.find_elements_by_xpath(f"{base_first_xpath}//ul/li")
    profile_data['profile_name'] = top_box_uls[0].text
    profile_data['connection_degree'] = top_box_uls[1].text.split('\n')[1]
    profile_data['profile_location'] = top_box_uls[3].text
    profile_data['profile_n_conn'] = top_box_uls[4].text.split(' ')[0]
    
    profile_data['profile_header'] = driver.find_element_by_xpath(f"{base_first_xpath}//h2").text
    
    about_xpath = "//div[@class='application-outlet ']//div[@class='profile-detail']/div/section/p/span"
    try:
        about_show_more_element = driver.find_element_by_xpath("//a[@id='line-clamp-show-more-button']")
    
        actions = ActionChains(driver)
        coordinates = about_show_more_element.location_once_scrolled_into_view # returns dict of X, Y coordinates
        driver.execute_script(f"window.scrollTo({coordinates['x']}, {coordinates['y']});")
    
        about_show_more_element.click()
    except Exception as e:
        print(e)
    about_me_lines = driver.find_elements_by_xpath(about_xpath)
    profile_data['profile_about_me'] = " ".join([line.text for line in about_me_lines])
    while check_for_see_more_buttons(driver):
        click_all_see_more_buttons(driver)
        
        
    base_metadata_xpath = '//section[@class="pv-profile-section pv-profile-section--reorder-enabled background-section artdeco-container-card ember-view"]'
    
    experience_dict = {}
    education_dict = {}
    lic_certif_dict = {}
    volunteer_dict = {}
    
    for section_index, section in enumerate(driver.find_elements_by_xpath(base_metadata_xpath + '/div')):
        section_title = driver.find_elements_by_xpath(base_metadata_xpath + '/div/section/header')[section_index].text
        
        section_id = driver.find_elements_by_xpath(base_metadata_xpath + '/div/section')[section_index].get_attribute('id')
        section_items = driver.find_elements_by_xpath(base_metadata_xpath + f'/div/section[@id="{section_id}"]/ul/li')
        item_ids = [item.get_attribute('id') for item in section_items]
        for item_index, item_id in enumerate(item_ids):
            item_xpath_base = base_metadata_xpath + f'/div/section[@id="{section_id}"]/ul/li[@id="{item_id}"]'
            item_dict = {}
            if section_title == 'Experience':
                expr_title = driver.find_element_by_xpath(item_xpath_base + '//h3').text
                expr_company = driver.find_element_by_xpath(item_xpath_base + '//p[@class="pv-entity__secondary-title t-14 t-black t-normal"]').text
                item_dict['title'] = expr_title
                item_dict['company'] = expr_company
                experience_dict[item_id] = item_dict
            if section_title == 'Education':
                item_dict['institution'] = driver.find_element_by_xpath(item_xpath_base + '//h3').text
                edu_p_list = driver.find_elements_by_xpath(item_xpath_base + '//p')
                
                if len(edu_p_list) >= 3:
                    item_dict['degree'] = edu_p_list[0].text.split('\n')[1] + ' ' + edu_p_list[1].text.split('\n')[1]
                    item_dict['years'] = edu_p_list[2].text.split('\n')[1]
                elif len(edu_p_list) == 1:
                    item_dict['years'] = edu_p_list[0].text.split('\n')[1]
                
                education_dict[item_id] = item_dict
                
    profile_data['experience'] = experience_dict
    profile_data['education'] = education_dict
        
    return profile_data

In [162]:
profile_url = 'https://www.linkedin.com/in/marcie-matthews-6812a22/'
profile_url = 'https://www.linkedin.com/in/ckirkup/'
li_profile = LinkedInProfile()
options = webdriver.FirefoxOptions()
#options.add_argument('--headless')
driver = webdriver.Firefox(options=options)


sign_in(driver, li_profile)
scrape_profile(driver, profile_url)

i click
i click


{'profile_name': 'Chris Kirkup',
 'connection_degree': '1st',
 'profile_location': 'Greater Boston Area',
 'profile_n_conn': '441',
 'profile_header': 'Data Science Partnerships at nference',
 'profile_about_me': 'Bioinformatics student skilled in Genomics Data Analysis and Biotechnology Lab Techniques with interests in Data Science, Machine Learning and Systems Biology; graduating Spring 2020 with Bachelor of Science (B.S.) in Biology and Spring 2021 with Masters of Science (M.S.) in Bioinformatics from Northeastern University ',
 'experience': {'ember176': {'title': 'Data Science Partnerships',
   'company': 'nference'},
  'ember184': {'title': 'Functional Genomics/Bioinformatics Co-op',
   'company': 'Agenus'},
  'ember192': {'title': 'Immunomodulatory Drug Discovery Co-op',
   'company': 'Agenus'},
  'ember200': {'title': 'Clinical Care Technician',
   'company': 'Tufts Medical Center'},
  'ember210': {'title': 'Research Assistant',
   'company': 'Northeastern University'},
  'embe

In [163]:
close_driver(driver)